# 03 Scopes

As discussed in [02 Service Lifetimes](02_lifetimes.ipynb), services in a `ServiceContainer` can have a lifetime associated with them.
To briefly recap, service lifetimes can be defined as:
 - Transient: a new service instance is created each time it is retrieved
 - Scoped: a single instance is created per scope
 - Singleton: a single instance is created per `ServiceContainer`

Transient and Singleton services are fairly simple to understand, and don't require any additional mechanics to make use of them. However, Scoped services require a `Scope`.

Bear with me, this is going to be a bit of a wordy one...

**What is a Scope?**  
A scope acts as a boundary within which scoped services are instantiated and managed. Think of a scope as a distinct context or environment that controls the lifecycle of its scoped services. This ensures that within a given scope:
 - Scoped services are created once and reused whenever requested.
 - Transient and Singleton services behave consistently across scopes, adhering to their respective lifetimes.

**How Scopes Work**  
Each scope maintains its own set of scoped service instances. This means that scoped services within one scope are isolated from those in another, preventing unintended sharing or conflicts.

When a new scope is created, it inherits all the service registrations from the parent `ServiceContainer`. However, for scoped services, the scope ensures that new instances are provided within its boundary while respecting singleton and transient lifetimes.

Once a scope is exited, it is typical that all scoped service instances within it are disposed. This automatic cleanup helps manage resources efficiently and avoids memory leaks.

Scopes operate with a stack. When you enter into a new scope, it is pushed to the top of the stack. And conversly, when you exit from a scope, it is popped from the top of the stack. When you retrieve a scoped service from the `ServiceCollection`, it will retrieve it from the top of the scope stack.  
In PyJudo, there is no "base" or "global" scope, you need to explicitly enter a new scope. Trying to retrieve a service without entering a scope will throw an exception*.

**When to use Scopes**  
Scoped services are ideal in scenarios where you need:
 - "Per-request" Lifetimes  
   For example, in web applications it is typical to create a new scope for each HTTP request. This ensures that services are consistent throughout the handling of that request but isolated from others.

 - "Operational" Services
   When services hold resources that should be released after a specific operation or context. Scoped lifetimes ensure isolation from other (sometimes concurrent) operations and provide a post-operation disposal mechanism.


> **\* NOTE**  
> It is debatable whether there should be a "global" scope, but I'm of the opinion that this is a poor design.  
>  
> Services which are scoped imply they have an expiration and should be disposed of. A "global" scope would not be disposed of until the container is disposed, meaning scoped services resolved in the "global" scope are effectively implicit singletons.  
>  
> I think it's better to explicitly define that behaviour to prevent accidental misuse; thus, scoped services cannot be resolved unless within an explicitly defined scope.

Lets take a look at some examples to better understand scopes and scoped services...

In [20]:
from abc import ABC, abstractmethod

from pyjudo import ServiceContainer

In [21]:
# Interfaces
class IFoo(ABC):
    text: str

    @abstractmethod
    def foo(self): ...

class IBar(ABC):
    @abstractmethod
    def bar(self): ...


# Implementations
class Foo(IFoo):
    def __init__(self, text: str = "default text"):
        self.text = text

    def foo(self):
        print(self.text)

class Bar(IBar):
    def __init__(self, foo: IFoo):
        self.foo = foo

    def bar(self):
        print(f"My Foo says: {self.foo.text}")


In [22]:
# Create a container and register the services
container = ServiceContainer()

container.add_scoped(IFoo, Foo)
container.add_transient(IBar, Bar)

Note that we have registered `IFoo` as a scoped service.

In [23]:
# Simple scope
with container.create_scope() as scope:
    bar = scope.get(IBar)
    bar.bar()


My Foo says: default text


This is the most basic use of a scope. Enter into a new scope, retrieve our service, and utilise it within the scope. EZ PZ.

In [24]:
from pyjudo.exceptions import ServiceResolutionError

try:
    foo = container.get(IFoo)
except ServiceResolutionError as e:
    print(f"Error: {e}")

Error: Service 'IFoo' is scoped but no scope was provided.


As previously mentioned, scoped services require an explicit scope to be retrieved. Here we get a `ServiceResolutionError` because we are trying to retrieve `IFoo` outside of any scopes.

In [25]:
# Nested scope
with container.create_scope() as scope:
    foo_outer = scope.get(IFoo)
    foo_outer.text = "Hello from outer scope"
    
    with container.create_scope() as nested_scope:
        foo_inner = nested_scope.get(IFoo)
        foo_inner.foo()

    foo_outer.foo()

default text
Hello from outer scope


This example makes use of scope stacks. The `IFoo` in the outer scope is modified, but the inner `IFoo` displays the original default text.

All of the examples so far have used the syntax:
```python
with container.create_scope() as scope:
    ... = scope.get(...)
```

This gives our scope an identifier, in this case `scope`. Technically, this is not required. You can directly use the `ServiceContainer`, because it will always try to retrieve scoped services from the top of the scope stack.

It is just as valid to do the following:

In [26]:
# Unnamed nested scope
with container.create_scope():
    foo_outer = container.get(IFoo)
    foo_outer.text = "Hello from outer scope"
    
    with container.create_scope():
        foo_inner = container.get(IFoo)
        foo_inner.foo()

    foo_outer.foo()

default text
Hello from outer scope


Personally, I prefer the named style and will use it going forward. I think it is more readable, particularly when the context body gets long.

At this point, you might be wondering "hey, what if i access the `foo_inner` outside of the scope it was created in" (at least if you're anything like me...)

In [27]:
# Out of scope service use
with container.create_scope() as scope:
    foo_outer = scope.get(IFoo)
    foo_outer.text = "Hello from outer scope"
    
    with container.create_scope() as nested_scope:
        foo_inner = nested_scope.get(IFoo)
    
    foo_inner.foo()
    foo_outer.foo()

default text
Hello from outer scope


Well, the answer is pretty much "nothing - it does what you'd expect... kinda".  

In the example above we did as usual, retrieved a `foo_outer` and a `foo_inner`, but this time accessed them both in the outer scope. And `foo_inner` printed as it did previously...  
Well, that's kinda what we expected, but did't that big long winded text block at the start of this example talk about "disposing" and "isolation"? We've just blown straight through that "isolation"... 

In some cases, the displayed behaviour may be intended. You may want to control creating `IFoo` instances in a scope to be used later outwith the scope. We're also in "Python" land, where rules are not so much rules, but small fences to be tripped over.  

It is however typical that scoped services are disposed of when leaving the scope they were created in. In order to do that, we need to implement a `dispose` method on any scoped service, so that the `ServiceContainer` knows what to do with each service when the scope is exited.

In [28]:
class DisposableFoo(IFoo):
    def __init__(self, text: str = "default text"):
        self.text = text

    def foo(self):
        print(self.text)

    def dispose(self):
        self.text = "IM DISPOSED DONT USE ME"

In [29]:
# Register the new DisposableFoo
container.unregister(IFoo)

container.add_scoped(IFoo, DisposableFoo)

In [30]:
# Out of scope service use with disposable
with container.create_scope() as scope:
    foo_outer = scope.get(IFoo)
    foo_outer.text = "Hello from outer scope"
    
    with container.create_scope() as nested_scope:
        foo_inner = nested_scope.get(IFoo)
        foo_inner.foo()
    
    foo_inner.foo()
    foo_outer.foo()

default text
IM DISPOSED DONT USE ME
Hello from outer scope


This time when we accessed our `DisposableFoo` outside its scope, it printed the updated text set in `dispose()`. So, if our scoped service has a `dispose()` method, the `ServiceContainer` will automatically call it on each scoped service when it leaves a scope.

This is useful for updating a services state when a scope exits, but it doesnt really prevent any accidental continued use of the service outwith the scope (unless you implement it).  

PyJudo provides an `IDisposable` abstract class, which will do just that.

In [31]:
from pyjudo.disposable import IDisposable

class DisposableFoo_v2(IFoo, IDisposable):
    def __init__(self, text: str = "default text"):
        self.text = text

    def foo(self):
        print(self.text)

    def do_dispose(self):
        self.text = "IM DISPOSED DONT USE ME"

In [32]:
# Register the new-new DisposableFoo
container.unregister(IFoo)

container.add_scoped(IFoo, DisposableFoo_v2)

In [33]:
# Out of scope service use with disposable v2

from pyjudo.exceptions import ServiceDisposedError

with container.create_scope() as scope:
    foo_outer = scope.get(IFoo)
    foo_outer.text = "Hello from outer scope"
    
    with container.create_scope() as nested_scope:
        foo_inner = nested_scope.get(IFoo)
        foo_inner.foo()
    
    try:
        foo_inner.foo()
    except ServiceDisposedError as e:
        print(f"Error: {e!r}")
    foo_outer.foo()

default text
Error: ServiceDisposedError('Object is disposed and cannot be used.')
Hello from outer scope


That's more like it. When we tried to access the `foo_inner` outside of it's scope, we get a `ServiceDisposedError`. This prevents services being used outwith their scope, ensuring their isolation.

And with that, I think that mostly covers scoped services. I hope that gave you some useful information on scoped service lifetimes and how they are implemented in PyJudo!